In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
batch_size = 256
num_workers = 4
lr = 0.001
num_epochs = 200

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm

In [4]:
class Residual(nn.Module):
    def __init__(self, in_channels, out_channels, use_1x1conv=False, stride=1):
        super(Residual, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride, 1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, 1, 1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(in_channels, out_channels, 1, stride)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        out = self.bn2(self.conv2(F.relu(self.bn1(self.conv1(x)))))
        if self.conv3:
            x = self.conv3(x)
        return out + x

In [5]:
def residual_block(in_channels, out_channels, num_repeat, first_block=False):
    if first_block:
        assert in_channels == out_channels
    blk = []
    for i in range(num_repeat):
        if i == 0 and not first_block:
            blk.append(Residual(in_channels, out_channels, use_1x1conv=True, stride=2))
        else:
            blk.append(Residual(out_channels, out_channels))
    return nn.Sequential(*blk)

In [6]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        blk = []
        blk.append(nn.Sequential(
            nn.Conv2d(1, 64, 7, 2, 3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)))
        blk.append(residual_block(64, 64, 2, True))
        blk.append(residual_block(64, 128, 2))
        blk.append(residual_block(128, 256, 2))
        blk.append(residual_block(256, 512, 2))
        blk.append(nn.AdaptiveAvgPool2d(1))
        self.blk = nn.Sequential(*blk)
        self.fc = nn.Linear(512, 10)
    
    def forward(self, x):
        b, c, h, w = x.shape
        return self.fc(self.blk(x).view(b, -1))

In [7]:
trans = []
trans.append(transforms.Resize((96, 96)))
trans.append(transforms.ToTensor())
transform = transforms.Compose(trans)
mnist_train = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=True, download=True, transform=transform)
mnist_test = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=False, download=True, transform=transform)
print(len(mnist_train), len(mnist_test))
train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)

60000 10000


In [8]:
net = ResNet().cuda()
optimizer = torch.optim.Adam(net.parameters(), lr)
loss = torch.nn.CrossEntropyLoss()

In [9]:
def train_FashionMNIST(net, train_iter, optimizer):
    train_loss = 0.0
    train_acc = 0.0
    train_num = 0
    
    for X, y in train_iter:
        X = X.cuda()
        y = y.cuda()
        y_hat = net(X)
        l = loss(y_hat, y)
        optimizer.zero_grad()
        # net.zero_grad()
        l.backward()
        optimizer.step()
        
        train_loss += l.item()
        train_acc += (y_hat.argmax(dim=1) == y).sum().item()
        train_num += y.shape[0]
    
    train_loss /= train_num
    train_acc /= train_num
    print('train loss: %.4f, train acc: %.3f' % (train_loss, train_acc))

In [10]:
def test_FashionMNIST(net, test_iter):
    test_acc = 0.0
    test_num = 0
    
    for X, y in test_iter:
        X = X.cuda()
        y = y.cuda()
        y_hat = net(X)
        test_acc += (y_hat.argmax(dim=1) == y).sum().item()
        test_num += y.shape[0]
    
    test_acc /= test_num
    print('test acc: %.3f' % (test_acc))

In [ ]:
for i in range(num_epochs):
    print(f'epoch: {i}')
    train_FashionMNIST(net, train_iter, optimizer)
    test_FashionMNIST(net, test_iter)
    print('----------------')

epoch: 0
train loss: 0.0019, train acc: 0.831
test acc: 0.877
----------------
epoch: 1
train loss: 0.0011, train acc: 0.896
test acc: 0.900
----------------
epoch: 2
train loss: 0.0009, train acc: 0.913
test acc: 0.901
----------------
epoch: 3
train loss: 0.0008, train acc: 0.921
test acc: 0.913
----------------
epoch: 4
train loss: 0.0008, train acc: 0.930
test acc: 0.919
----------------
epoch: 5
train loss: 0.0007, train acc: 0.935
test acc: 0.923
----------------
epoch: 6
train loss: 0.0006, train acc: 0.940
test acc: 0.922
----------------
epoch: 7
train loss: 0.0006, train acc: 0.945
test acc: 0.918
----------------
epoch: 8
train loss: 0.0005, train acc: 0.949
test acc: 0.923
----------------
epoch: 9
train loss: 0.0005, train acc: 0.954
test acc: 0.923
----------------
epoch: 10
train loss: 0.0005, train acc: 0.956
test acc: 0.922
----------------
epoch: 11
train loss: 0.0004, train acc: 0.961
test acc: 0.925
----------------
epoch: 12
train loss: 0.0004, train acc: 0.964
tes

In [ ]:
# Adam: lr=0.001, epoch 49, test_acc=0.920, batch_size=256
# Adam: lr=0.005, epoch 0, test_acc=0.100, batch_size=256
# Adam: lr=0.0005, epoch 25, test_acc=0.918, batch_size=256